<a href="https://colab.research.google.com/github/swapnil14g/amaz/blob/main/amaz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/src/')

# Import custom modules
from utils import download_images, parse_string
from constants import entity_unit_map, allowed_units

csv_file_path = '/content/drive/MyDrive/Colab Notebooks/dataset/train.csv'
download_folder = "Images"

# Create directory if it doesn't exist
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Load the CSV
df = pd.read_csv(csv_file_path)
image_links = df['image_link'].tolist()

# Download images
download_images(image_links, download_folder, allow_multiprocessing=True)
# Set up paths
DATASET_FOLDER = '/content/drive/MyDrive/Colab Notebooks/dataset/'
IMAGE_FOLDER = '/content/drive/MyDrive/Colab Notebooks/Images/'
OUTPUT_FILE = 'test_out.csv'

# Load data
train_df = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

# Download images (uncomment when ready to download)
download_images(train_df['image_link'], IMAGE_FOLDER)
download_images(test_df['image_link'], IMAGE_FOLDER)

# Custom dataset class
class ProductImageDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, os.path.basename(self.dataframe.iloc[idx]['image_link']))
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        entity_name = self.dataframe.iloc[idx]['entity_name']
        if 'entity_value' in self.dataframe.columns:
            entity_value = self.dataframe.iloc[idx]['entity_value']
            number, unit = parse_string(entity_value)
            return image, entity_name, number, unit
        else:
            return image, entity_name

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = ProductImageDataset(train_df, IMAGE_FOLDER, transform=transform)
test_dataset = ProductImageDataset(test_df, IMAGE_FOLDER, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define model
class EntityExtractionModel(nn.Module):
    def __init__(self, num_classes):
        super(EntityExtractionModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        self.fc1 = nn.Linear(num_ftrs, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.fc3 = nn.Linear(256, 1)  # for numeric value

    def forward(self, x):
        x = self.resnet(x)
        x = torch.relu(self.fc1(x))
        unit = self.fc2(x)
        value = self.fc3(x)
        return unit, value

# Initialize model, loss, and optimizer
num_classes = len(allowed_units)
model = EntityExtractionModel(num_classes)
criterion_unit = nn.CrossEntropyLoss()
criterion_value = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for images, entity_names, numbers, units in tqdm(train_loader):
        images = images.to(device)
        numbers = numbers.to(device).float()
        units = torch.tensor([list(allowed_units).index(u) for u in units]).to(device)

        optimizer.zero_grad()
        unit_pred, value_pred = model(images)
        loss_unit = criterion_unit(unit_pred, units)
        loss_value = criterion_value(value_pred.squeeze(), numbers)
        loss = loss_unit + loss_value
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# Prediction on test set
model.eval()
predictions = []

with torch.no_grad():
    for images, entity_names in tqdm(test_loader):
        images = images.to(device)
        unit_pred, value_pred = model(images)
        unit_pred = torch.argmax(unit_pred, dim=1)
        unit_pred = [list(allowed_units)[i] for i in unit_pred.cpu().numpy()]
        value_pred = value_pred.squeeze().cpu().numpy()

        for value, unit in zip(value_pred, unit_pred):
            predictions.append(f"{value:.2f} {unit}")

# Create output file
test_df['prediction'] = predictions
test_df[['index', 'prediction']].to_csv(OUTPUT_FILE, index=False)

print(f"Output saved to {OUTPUT_FILE}")

# Run sanity check
os.system(f"python src/sanity.py --test_filename {os.path.join(DATASET_FOLDER, 'test.csv')} --output_filename {OUTPUT_FILE}")

 37%|███▋      | 97526/263859 [45:58<1:11:43, 38.65it/s]

Mounted at /content/drive
